In [14]:
import json
import os
from collections import Counter 
import random
import subprocess

In [10]:
random.seed(8)

In [3]:
genus = [f[:-5] for f in os.listdir('cooked_data') if f.endswith('.json')]

In [21]:
g = genus[0]

with open('cooked_data/{}.json'.format(g), 'r') as f:
    data = json.load(f)
    

while True:
    pair = random.choices(data, k=2)
    if pair[0]['area'] == pair[1]['area']:
        break

seq1, seq2 = subprocess.check_output(['alignment', pair[0]['coi5p'], pair[1]['coi5p']]).decode('utf-8').split('\n')[:2]

CPU times: user 1.88 ms, sys: 25.8 ms, total: 27.6 ms
Wall time: 238 ms


In [20]:
seq1, seq2

('ATTCTATATTTTATTTTTGGTATATGAGCTGGTATACTAGGAGCCTCCATAAGATTAATTATTCGTATAGAATTAGGATCAACAGGATCATTAATTGGAAATGATCAAATTTATAATAATTTAGTTACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGAAATTGGTTAGTACCATTAATACTTGGAGCTCCAGATATAGCATTTCCACGCATAAATAATATAAGATTTTGATTACTACCCCCTTCCTTATTTCTTTTAATTAATAGAAGATTTATTAATCAAGGAGTAGGAACTGGATGAACTGTTTACCCTCCCTTATCTTTAAATGTTAATCATGAAGGAATATCTGTTGACATAGCAATTTTTTCTTTACATATAGCAGGGATATCTTCTATTATAGGATCAATTAATTTTATTTCAACTATTTTAAATATATACCCCATAAACTCTAAATTAGATCAAATCACTTTATTTACTTGATCAATTTTAATTACAACAATTTTACTTTTATTAGCTGTACCAGTTTTAGCTGGA------------------------------------------------------------------------',
 'ATTCTATATTTTATTTTTGGTATATGAGCTGGTATACTAGGAGCCTCCATAAGATTAATTATTCGTATAGAATTAGGATCAACAGGATCATTAATTGGAAATGATCAAATTTATAATAATTTAGTTACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTATTATAATTGGAGGATTTGGAAATTGGTTAGTACCATTAATACTTGGAGCTCCAGATATAGCATTTCCACGCATAAATAATATAAGATTTTGATTACTACCCCCTTCCTTATTTCTTTTAATTAATAGAAGATTTATTAATCAAGGAGTAGGAACTGGATGAACTGTTTACCCTCCCTTATCTTTAAAT